In [97]:
from dash import Dash, html, dcc, Input, Output, dash_table
import dash
import pandas as pd
import plotly.express as px

import pandas as pd
from os import path
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [2]:
nlp_file_path = f"CroLyrics_data/nlp_all.csv"
df = pd.read_csv(nlp_file_path)
df_main_words = df[df.upos.isin(["ADV", "PRON", "ADJ", "NOUN", "VERB", "PROPN"])]
df_main_words.reset_index(drop=True, inplace=True)

df_unique_lemmas = df_main_words.drop_duplicates(subset=["lemma", "Song_ID"]).reset_index(drop=False)
df_unique_lemmas["Artist_ID"] = df_unique_lemmas.Song_ID.str.split("_").map(lambda x: x[0])
locations = df_unique_lemmas[(df_unique_lemmas["ner"].isin(["B-LOC", "I-LOC"])) & (df_unique_lemmas["upos"]=="PROPN")]#["lemma"]#.value_counts()
names = df_unique_lemmas[(df_unique_lemmas["ner"].isin(["B-PER", "I-PER"])) & (df_unique_lemmas["upos"]=="PROPN")]#["lemma"]#.value_counts()
organizations = df_unique_lemmas[(df_unique_lemmas["ner"].isin(["B-ORG", "I-ORG"])) & (df_unique_lemmas["upos"]=="PROPN")]#["lemma"]#.value_counts()
nouns = df_unique_lemmas[(df_unique_lemmas.upos=="NOUN")]#["lemma"]#.value_counts()
verbs = df_unique_lemmas[(df_unique_lemmas.upos=="VERB")]#["lemma"]#.value_counts()
adjectives = df_unique_lemmas[(df_unique_lemmas.upos=="ADJ")]#["lemma"]#.value_counts()
adverbs = df_unique_lemmas[(df_unique_lemmas.upos=="ADV")]#["lemma"]#.value_counts()


In [5]:
def get_artist_name(artist_code):
    conn = sqlite3.connect('CroLyrics_data/info.db')
    c = conn.cursor()
    c.execute("SELECT name FROM artists WHERE code=:code", {'code': artist_code})
    return c.fetchone()

def get_number_of_songs(artist_code):
    conn = sqlite3.connect('CroLyrics_data/info.db')
    c = conn.cursor()
    c.execute("SELECT number_of_songs FROM artists WHERE code=:code", {'code': artist_code})
    return c.fetchone()

In [71]:
def generate_table(df, word):
    return html.Div(children = [html.H6(children=f'List of songs mentioning {word}'),
    	dash_table.DataTable(
            id='datatable-interactivity',
            columns=[
            {"name": i, "id": i, "deletable": False, "selectable": False, "presentation": 'markdown'} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable=False,
        row_deletable=False,
        page_action="native",
        page_current= 0,
        page_size= 10,
    )])




In [116]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div([
    html.Div(className='row', children=[
        html.Div(className='three columns', children = [
                dcc.Dropdown(
                    ['Location', 'Name', "Organization", "Noun", "Verb", "Adjective", "Adverb"],
                    'Location',
                    id='barplot-subgroup',
                ),
                ], style={'width': '30%', 'display': 'inline-block'}),
        html.Div(className='three columns', children = [
            dcc.RadioItems(
                ['Absolute', 'Percentage'],
                'Absolute',
                id='abs-vs-perc-artists',
                labelStyle={'width': '30%', 'display': 'inline-block'}
            )
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '0% 0%'})
    ], style={'padding': '5% 5% 0% 5%'}),

    html.Div(className='row', children=[
        html.Div(className='three columns', children = [
            dcc.Graph(
                id='barplot-most-common',
                clickData={'points': [{'text': 'Hrvatska'}]}
            )
        ], style={'display': 'inline-block', 'width': '30%'}),

        html.Div(className='three columns', children = [
            dcc.Graph(id='barplot-artists'),
        ], style={'display': 'inline-block', 'width': '30%'}),

        html.Div(className='three columns', id='songs-table', children=[], style={'display': 'inline-block', 'width': '30%'})

    ], style={'padding': '0% 5%'}),

   
    html.Div([dcc.Store(id='intermediate-df')])
])



@app.callback(
    Output('intermediate-df', 'data'),
    Input('barplot-subgroup', 'value'))
def get_subdf(subgroup):
    if subgroup == "Location": dataframe = locations
    elif subgroup == "Name": dataframe = names
    elif subgroup == "Organization": dataframe = organizations
    elif subgroup == "Noun": dataframe = nouns
    elif subgroup == "Verb": dataframe = verbs
    elif subgroup == "Adjective": dataframe = adjectives
    elif subgroup == "Adverb": dataframe = adverbs
    return dataframe.to_json(date_format='iso', orient='split'), subgroup



@app.callback(
    Output('barplot-most-common', 'figure'),
    Input('intermediate-df', 'data'))
def update_graph(subgroup_df):
    dataframe = pd.read_json(subgroup_df[0], orient='split')
    value_counts = dataframe.lemma.value_counts()   
    fig = px.bar(
                x=np.flip(value_counts[0:10].values), text=np.flip(value_counts[0:10].index), 
               orientation='h')
    
    fig.update_traces(hovertemplate='Word: %{text} <br>Number of songs: %{x}')
    
    fig.update_yaxes(title=subgroup_df[1], showticklabels=False)
    fig.update_xaxes(title="Number of songs")
    fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
    return fig

def create_artists_plot(artists, counts, total_counts, abs_or_perc, lemma):
    if abs_or_perc == "Absolute": 
        x = counts
        text = artists
        x_label = "Number of songs"
        hover = 'Artist: %{text} <br>Number of songs: %{x}'
    elif abs_or_perc == "Percentage": 
        sorted_perc = [[x, y] for x, y in sorted(zip(counts/total_counts, artists), reverse=True)]
        x = [s[0]*100 for s in sorted_perc]
        text = [s[1] for s in sorted_perc]
        x_label = "Percentage of songs"
        hover = 'Artist: %{text} <br>% of songs: %{x}'

    fig = px.bar(x=np.flip(x[0:10]), text=np.flip(text[0:10]), orientation='h', title=f"Top 10 artists mentioning the word {lemma}")
    fig.update_traces(hovertemplate=hover)
    fig.update_yaxes(title="Artist", showticklabels=False)
    fig.update_xaxes(title=x_label)
    fig.update_layout(margin=dict(l=20, r=20, t=25, b=20))


    return fig





@app.callback(
    Output('barplot-artists', 'figure'),
    Input('barplot-most-common', 'clickData'),
    Input('abs-vs-perc-artists', 'value'),
    Input('intermediate-df', 'data'))
def update_artists_plot(clickData, abs_or_perc, subgroup_df):
    dataframe = pd.read_json(subgroup_df[0], orient='split')
    
    ctx = dash.callback_context
    input_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if input_id == "barplot-most-common" or input_id == "abs-vs-perc-artists": lemma = clickData['points'][0]['text']
    elif input_id == "intermediate-df": lemma = dataframe.lemma.value_counts().index[0]

    value_counts_artist = dataframe[dataframe.lemma == lemma].Artist_ID.value_counts()
    artists = value_counts_artist.index.map(lambda x: get_artist_name(x)[0])
    counts = value_counts_artist.values
    total_counts = [get_number_of_songs(a)[0] for a in value_counts_artist.index]
        
    return create_artists_plot(artists, counts, total_counts, abs_or_perc, lemma)

  


@app.callback(
    Output('songs-table', 'children'),
    Input('barplot-most-common', 'clickData'),
    Input('intermediate-df', 'data'))
def update_songs_table(clickData, subgroup_df):
    dataframe = pd.read_json(subgroup_df[0], orient='split')
    
    ctx = dash.callback_context
    input_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if input_id == "barplot-most-common": lemma = clickData['points'][0]['text']
    elif input_id == "intermediate-df": lemma = dataframe.lemma.value_counts().index[0]
        
    songs_ids = dataframe[dataframe.lemma == lemma].Song_ID.values
    all_data = []
    conn = sqlite3.connect('CroLyrics_data/info.db')
    c = conn.cursor()
    for i in range(0, len(songs_ids), 999):
        placeholder= '?'
        placeholders= ', '.join(placeholder for unused in songs_ids[i:i+999])
        query= 'SELECT name, artist_name, views, url FROM songs_info WHERE code IN (%s)' % placeholders
        c.execute(query, songs_ids[i:i+999])
        all_data.extend(c.fetchall())

    df_songs = pd.DataFrame(all_data)
    df_songs.columns = ["Song", "Artist", "Views", "Lyrics Url"]
    df_songs["Song"] = "[" + df_songs["Song"] + "](" + df_songs["Lyrics Url"] + ")"
    df_songs = df_songs.drop("Lyrics Url", axis=1)
    df_songs = df_songs.sort_values("Views", ascending=False)

    conn.close()
    return generate_table(df_songs, lemma)


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

In [79]:
    lemma = "ljubav"
    value_counts_artist = df_unique_lemmas[df_unique_lemmas.lemma == lemma].Artist_ID.value_counts()
    
    artists = value_counts_artist.index.map(lambda x: get_artist_name(x)[0])
    counts = value_counts_artist.values
    total_counts = [get_number_of_songs(a)[0] for a in value_counts_artist.index]

In [81]:
value_counts_artist

u5IcBEql    102
J8Cff2I3     71
QUgrhjBj     69
Fv0Kbouf     67
538Exp8s     66
rLIJvpNx     63
l6V8OOIw     58
vmjQPyJ5     58
zT3Xu5sD     56
vpm1INqf     43
RH/jD1hv     41
CDOhygHn     39
CWc8m1qi     39
+WaQ+Bqb     38
6QUk9BaE     37
FJdcVEAU     36
nDEq9hkG     36
42+RWAHa     34
/kSKwa9Q     33
D7roOOt+     33
Wm5tG4b5     32
pltir+OL     31
dPHel+/u     30
Xp4QBL1e     29
vOntpwRZ     28
EOkBEcEU     28
3xr7k3c7     27
PZ5KqfC1     27
8T5eTuU5     26
wNGUjqQ7     26
4XHKfXTU     25
O93DTxfd     25
LC92qBcX     24
JWLKah/6     23
hcEyZCpA     22
TkHYG/uZ     22
/gYDFxB8     22
+RXzsM0Y     22
hluW+yB/     21
b1GNJHQO     20
RVoUr7Lb     18
MZPizswt     13
Wal+tKVc     12
4wxdBzAi     10
r01Z81Es     10
d/vUPGnO      9
CXljSNTE      7
JUdMSWu4      7
0X3VJo7g      6
igNyBvbR      5
0iPNl+uu      4
zdPtejCw      3
Name: Artist_ID, dtype: int64